In [ ]:
#!pip install tensorflow
#!pip install tqdm
#!pip install --upgrade music21
#!pip install pandas
#!pip install glob
#!pip install KERAS


In [12]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.python.ops import control_flow_ops
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [13]:
notes = []
for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [14]:
n_vocab=len(set(notes))

In [15]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [1]:
print(len(notes))

NameError: name 'notes' is not defined

In [9]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger-singleMidi.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=10, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
4116/4116 [==============================] - ETA: 3111s - loss: 4.34 - ETA: 3100s - loss: 4.38 - ETA: 3069s - loss: 4.32 - ETA: 3023s - loss: 4.35 - ETA: 3001s - loss: 4.34 - ETA: 3022s - loss: 4.29 - ETA: 3005s - loss: 4.30 - ETA: 2990s - loss: 4.29 - ETA: 3004s - loss: 4.26 - ETA: 2967s - loss: 4.28 - ETA: 2920s - loss: 4.31 - ETA: 2898s - loss: 4.30 - ETA: 2884s - loss: 4.31 - ETA: 2852s - loss: 4.29 - ETA: 2817s - loss: 4.27 - ETA: 2793s - loss: 4.29 - ETA: 2752s - loss: 4.29 - ETA: 2726s - loss: 4.28 - ETA: 2691s - loss: 4.27 - ETA: 2658s - loss: 4.26 - ETA: 2618s - loss: 4.25 - ETA: 2569s - loss: 4.26 - ETA: 2515s - loss: 4.26 - ETA: 2467s - loss: 4.26 - ETA: 2413s - loss: 4.26 - ETA: 2366s - loss: 4.26 - ETA: 2320s - loss: 4.25 - ETA: 2280s - loss: 4.26 - ETA: 2235s - loss: 4.26 - ETA: 2176s - loss: 4.26 - ETA: 2125s - loss: 4.26 - ETA: 2073s - loss: 4.25 - ETA: 2020s - loss: 4.26 - ETA: 1961s - loss: 4.25 - ETA: 1906s - loss: 4.25 - ETA: 1847s - loss: 4.26 - ETA: 179

In [17]:
model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [18]:
# Load the weights to each node
model.load_weights('weights-improvement-99-2.4940-bigger-singleMidi.hdf5')

ValueError: Dimension 1 in both shapes must be equal, but are 2048 and 1024 for 'Assign_12' (op: 'Assign') with input shapes: [1,2048], [1,1024].

In [ ]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
pattern=list(pattern)
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output_500.mid')